In [19]:
import argparse
import pandas as pd
import numpy as np
import scipy.io
import nibabel.freesurfer.mghformat as mgh
import h5py
import os
from sklearn.cross_decomposition import PLSRegression
from fracridge import FracRidgeRegressor
import scipy.stats as stats

In [2]:
oakzfs_stem = '/sni-storage/kalanit/Projects/Dawn/NSD/'

UTILS_PATH = oakzfs_stem + 'code/fit_pipeline/utils/'
BETA_PATH = oakzfs_stem + 'local_data/processed/organized_betas/'
STIM_PATH = oakzfs_stem + 'data/nsddata_stimuli/stimuli/nsd/'
NSDDATA_PATH = oakzfs_stem + 'data/nsddata/'
FS_PATH = oakzfs_stem + 'local_data/freesurfer/'
FEATS_PATH = oakzfs_stem + 'code/fit_pipeline/models/features/'
RESULTS_PATH = oakzfs_stem + 'results/'


In [3]:

# how many times each image was shown
N_REPEATS = 3

# layer names for different models
ALEXNET_LAYERS = ['conv1', 'conv2', 'conv3', 'conv4', 'conv5', 'fc6', 'fc7']

In [4]:
import sys
sys.path.append(UTILS_PATH)

from rsm_utils import get_ROI_data
import regression_utils as rutils

In [5]:
subj = "02"
hemi = "rh"
roi = "streams_shrink10"
sub_roi = "ventral"
sub_roix = 5

In [6]:
def get_indices(subj):
    
    order = scipy.io.loadmat(BETA_PATH + 'datab3nativesurface_subj' + subj)
    data = pd.read_csv(NSDDATA_PATH + 'ppdata/subj' + subj + '/behav/responses.tsv', sep='\t')
    expdesign = scipy.io.loadmat(NSDDATA_PATH + 'experiments/nsd/nsd_expdesign.mat')
    
    #73KIDs
    all_ids = np.array(data['73KID'])
    vals, idx_start, count = np.unique(all_ids, return_counts=True, return_index=True)
    which_reps = vals[count == N_REPEATS]
    mask_3reps = np.isin(all_ids,which_reps)
    id_nums_3reps = np.array(data['73KID'])[mask_3reps]
    rep_vals = np.unique(id_nums_3reps) #sorted version of beta order
    
    #how the betas are ordered (using COCO 73K id numbers)
    beta_order_in_73Kids = all_ids[order['allixs'][0]-1]-1 #-1 to convert from matlab to python indexing
    
    # shared (i.e. validation) IDS (but include all potential shared reps for the subj, not min across subjs)
    sharedix = expdesign['sharedix'][0]
    validation_mask = np.isin(rep_vals,sharedix)

    return beta_order_in_73Kids, validation_mask

In [7]:
subj_keys = ["01", "05", "07"] #only use subjects with all 10k trials

In [10]:
target_betas = h5py.File(BETA_PATH + 'datab3nativesurface_subj' + subj +'_'+ hemi + '_betas.hdf5','r') 
beta_order_in_73Kids, validation_mask = get_indices(subj)

In [11]:
# get ROI data to pick which voxels to fit
mgh_file = mgh.load(FS_PATH + '/subj' + subj + '/' + hemi + '.' + roi + '.mgz')
streams = mgh_file.get_fdata()[:,0,0]

#trim and sort betas
stream_betas = target_betas['betas'][:,streams == sub_roix]
indx=beta_order_in_73Kids.argsort(axis=0)
sorted_betas = stream_betas[indx,:]

In [12]:
stream_betas.shape

(10000, 9051)

In [13]:
ordered_validation_betas = sorted_betas[validation_mask,:]

In [14]:
ordered_source_betas = {}
for s in subj_keys:
    print(s)
    source_betas = h5py.File(BETA_PATH + 'datab3nativesurface_subj' + s + '_'+ hemi + '_betas.hdf5','r') 
    beta_order_in_73Kids_source, validation_mask_source = get_indices(s)
    
    # get ROI data to pick which voxels to fit
    mgh_file = mgh.load(FS_PATH + '/subj' + s + '/' + hemi + '.' + roi + '.mgz')
    source_streams = mgh_file.get_fdata()[:,0,0]

    #trim and sort betas
    source_stream_betas = source_betas['betas'][:,source_streams == sub_roix]
    source_indx=beta_order_in_73Kids_source.argsort(axis=0)
    source_sorted_betas = source_stream_betas[source_indx,:]
    
    ordered_source_betas[s] = source_sorted_betas[validation_mask_source,:]

01
05
07


In [15]:
#all splits for regression training
num_splits = 1 #average over two random splits
all_splits = rutils.get_splits(data=ordered_validation_betas,
                               split_index=0,
                               num_splits=num_splits,
                               num_per_class_test = 200,
                               num_per_class_train = 800)

In [16]:
#use a different split to cv the number of components?
num_splits = 1 #average over two random splits
cv_splits = rutils.get_splits(data=ordered_validation_betas,
                               split_index=0,
                               num_splits=num_splits,
                               num_per_class_test = 200,
                               num_per_class_train = 800)

In [17]:
ordered_source_betas["01"].shape

(1000, 7749)

In [30]:
all_resdict = {}
best_cs = {}

cs = [.047, .048, .049, .05, .053, .054, .055, .056, .075, .76, .0775, .079]
for s in subj_keys:   #for each subj ...
    
    print('evaluating %s' % s)
    feats = ordered_source_betas[s]
    
    win_res = 0
    for c in cs:
        #print('evaluating component %s' % c)

        cv_res = rutils.train_and_test_scikit_regressor(features=feats, 
                                                    labels=ordered_validation_betas,
                                                    splits=cv_splits,
                                                    model_class=FracRidgeRegressor,
                                                    model_args={'fracs': c,
                                                               'normalize': False},
                                                    feature_norm=False)
        if cv_res['test']['mean_rsquared'] > win_res:
            win_res = cv_res['test']['mean_rsquared']
            best_cs[s] = c
    
    print(best_cs[s])
    res = rutils.train_and_test_scikit_regressor(features=feats, 
                                                    labels=ordered_validation_betas,
                                                    splits=all_splits,
                                                    model_class=FracRidgeRegressor,
                                                    model_args={'fracs': best_cs[s], #winning num components
                                                                'normalize': False},
                                                    feature_norm=False)
            
    all_resdict[s] = res

evaluating 01
0.0775
evaluating 05
0.049
evaluating 07
0.055


In [32]:
rsquared_array = {}
for s in subj_keys:
    rsquared_array[s] = all_resdict[s]['test']['mean_rsquared_array']

In [33]:
#save to local data folder
h5f = h5py.File(RESULTS_PATH+'fits/subj'+ subj + '_' + hemi + '_' + roi + '_' + sub_roi + '_ridgeCV_othersubjs_fits.hdf5', 'w')
for k, v in rsquared_array.items():
    print(str(k))
    h5f.create_dataset(str(k), data=v)
h5f.close()

01
05
07


In [176]:
all_resdict = {}
best_cs = {}

cs = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for s in subj_keys:   #for each subj ...
    
    print('evaluating %s' % s)
    feats = ordered_source_betas[s]
    
    win_res = 0
    for c in cs:
        #print('evaluating component %s' % c)

        cv_res = rutils.train_and_test_scikit_regressor(features=feats, 
                                                    labels=ordered_validation_betas,
                                                    splits=cv_splits,
                                                    model_class=PLSRegression,
                                                    model_args={'n_components': c,
                                                                'scale': False},
                                                    feature_norm=False)
        if cv_res['test']['mean_rsquared'] > win_res:
            win_res = cv_res['test']['mean_rsquared']
            best_cs[s] = c
    
    print(best_cs[s])
    res = rutils.train_and_test_scikit_regressor(features=feats, 
                                                    labels=ordered_validation_betas,
                                                    splits=all_splits,
                                                    model_class=PLSRegression,
                                                    model_args={'n_components': best_cs[s], #winning num components
                                                                'scale': False},
                                                    feature_norm=False)
            
    all_resdict[s] = res

evaluating 01
9
evaluating 05
9
evaluating 07
9


In [180]:
rsquared_array = {}
for s in subj_keys:
    rsquared_array[s] = all_resdict[s]['test']['mean_rsquared_array']

In [29]:
np.mean(rsquared_array["01"])

NameError: name 'rsquared_array' is not defined

In [182]:
#save to local data folder
h5f = h5py.File(RESULTS_PATH+'fits/subj'+ subj + '_' + hemi + '_' + roi + '_' + sub_roi + '_othersubjs_fits.hdf5', 'w')
for k, v in rsquared_array.items():
    print(str(k))
    h5f.create_dataset(str(k), data=v)
h5f.close()

01
05
07
